In [149]:
#sample running selenium in instagram

#import drivers and librabries
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import urllib2
import json
import pandas as pd, numpy as np
import csv
from lxml import html,etree
from datetime import datetime
from datetime import date
import requests,re
import os,sys
import unicodecsv as csv
import argparse
import urllib3
import itertools


In [150]:

class HWHostelList:
    
    def _auto_url(self, locality,country,date_from,date_to,number_of_guests):
        try:
            result = 'https://www.hostelworld.com/search?search_keywords='+str(locality)+',%20'+str(country)+'&country='+str(country)+'&city='+str(locality)+'&date_from='+str(date_from)+'&date_to='+str(date_to)+'&number_of_guests='+str(number_of_guests)+''
        except requests.HTTPError:
            raise requests.HTTPError('Received non 200 status code from HostelWorld')
        except requests.RequestException:
            raise requests.RequestException
        else:
            return result
        
    def get_hwurl(self, url):
        try:
            browser = webdriver.Chrome('//GitHub/selenium/chromedriver')
            browser.get(url)
            #Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            source = browser.page_source    
        except Exception as e:
            raise e
        else:
            return source
            
    def parse_hostel(self, html, loc):
        data = []
        parser = etree.HTML(html)
        hostel_list = parser.xpath('//div[contains(@id,"fabResultsContainer")]')

        for hostel in hostel_list:
            XPATH_HOSTEL_NAME = './/a[contains(@href,"https://www.hostelworld.com/hosteldetails.php/")]//text()'
            XPATH_HOSTEL_LINK = './/a[contains(@href,"https://www.hostelworld.com/hosteldetails.php/")]//@href'
            XPATH_HOSTEL_RATING = './/div[contains(@class,"rating score orange") and not(contains(@class,"small"))]//text()'
            XPATH_HOSTEL_REVIEWS = './/div[contains(@class, "reviews")]//text()'
            XPATH_HOSTEL_PRICE = './/span[contains(@class,"price")]//text()[not(contains(.,"Dorms")) and not(contains(.,"Private"))]'

            raw_hostel_name = hostel.xpath(XPATH_HOSTEL_NAME)
            raw_hostel_link = hostel.xpath(XPATH_HOSTEL_LINK)
            raw_hostel_rating = hostel.xpath(XPATH_HOSTEL_RATING)
            raw_hostel_reviews = hostel.xpath(XPATH_HOSTEL_REVIEWS)
            raw_hostel_price = hostel.xpath(XPATH_HOSTEL_PRICE)

            hostel_name = raw_hostel_name if raw_hostel_name else None
            hostel_link = raw_hostel_link if raw_hostel_link else None
            hostel_rating = raw_hostel_rating if raw_hostel_rating else None
            hostel_reviews = raw_hostel_reviews if raw_hostel_reviews else None
            hostel_price = raw_hostel_price if raw_hostel_price else None

            data = pd.DataFrame(list(zip(hostel_name,hostel_link,hostel_rating, hostel_reviews, hostel_price)), columns=['hostel_name','hostel_link','hostel_rating', 'hostel_reviews','hostel_price']) 

            data['hostel_reviews'] = data['hostel_reviews'].str.replace(' Total Reviews','') 
            data['hostel_reviews'] = data['hostel_reviews'].str.replace(' Total Review','')
            data['locality'] = loc

        return data
    
    def write_csv(self, data):
        try:
            data.to_csv('hostelworld_hostel.csv', mode='a', index = False, sep=',', encoding='utf-8', header=False)
        except Exception as e:
            raise e
        
    def read_hostelworld(self, url, loc):
        try:
            last_page = self.get_pagelength(url)
            loop = 1
            while loop < last_page+1:
                if loop > 1:
                    new_url = url+'&page='+str(loop)
                    parse = self.get_hwurl(new_url)
                    data = self.parse_hostel(parse, loc)
                    self.write_csv(data)
                else:
                    parse = self.get_hwurl(url)
                    data = self.parse_hostel(parse, loc)
                    self.write_csv(data)
                loop+=1
                time.sleep(60)
        except Exception as e:
            raise e
            
    def get_pagelength(self, url):
        try:
            driver = webdriver.Chrome('//GitHub/selenium/chromedriver')
            driver.get(url)
            last_page = driver.find_element_by_xpath('.//div[contains(@class,"pagination-last")]')
            last_page.click()
            last_page_url = driver.current_url
        except Exception as e:
            raise e
        else:
            result = str(last_page_url).replace(url+'&page=',"")
            driver.close()
            return int(result)
        

In [151]:

w = HWHostelList()

locality ='Canggu'
country = 'Indonesia'
date_from = '2020-01-01'
date_to = '2020-01-03'
number_of_guests = '2'


url = w._auto_url(locality,country,date_from,date_to,number_of_guests)
w.read_hostelworld(url, locality)